In [ ]:
import pandas as pd
import awswrangler as awr
import openpyxl

In [77]:
sql_path = r"C:\Users\raphael.almeida\Documents\Processos\conjuntos_placas_ativos\sql\conjuntos_ativos.sql"
with open (sql_path, 'r') as file:
    query = file.read()

df = awr.athena.read_sql_query(query, database='silver')

removendo duplicatas

In [81]:
df_total = df.drop_duplicates(subset=['placa'])

# VALIDAÇÃO CONJUNTOS

data

In [29]:
today = pd.Timestamp.today().normalize()
yesterday = today - pd.Timedelta(1, unit='day')
friday = today - pd.Timedelta(3, unit='day')

if today.day == 0:
    date = friday
else:
    date = yesterday

print(date)

2025-07-14 00:00:00


df_validacao (dia anterior)

In [32]:
df['data_registro'] = pd.to_datetime(df['data_registro']).dt.normalize()
df_validacao = df[df['data_registro']==date]
df_validacao.value_counts(subset='cooperativa')

cooperativa
Viavante    54
Stcoop      12
Name: count, dtype: int64

df_validacao2 (dia 8) - dia que bateu exato (porquê?)

In [67]:
random_date = pd.to_datetime('2025-07-08').normalize()  
df_validacao2 = df[df['data_registro']==random_date]
df_validacao2.value_counts(subset='cooperativa')
#df_validacao2[df_validacao2['cooperativa']=='Viavante'].value_counts('status')

Series([], Name: count, dtype: int64)

df_validacao3 (dia aleatório)

In [39]:
random_date = pd.to_datetime('2025-07-08').normalize()  
df_validacao3 = df[df['data_registro']==random_date]
#df_validacao3.value_counts(subset='cooperativa')
df_validacao3[df_validacao3['cooperativa']=='Viavante'].value_counts('status')

status
ATIVO          65
NOVO           25
SUBSTITUÍDO     1
Name: count, dtype: int64

# GERANDO PLACAS ATIVAS

In [82]:
df_total[df_total['status']=='ATIVO'].count()

df_total['placa'].nunique()

32242

### PLACAS POR EMPRESA

In [86]:
placas_totais = df_total['placa'].nunique()
placas_segtruck = df_total[df_total['empresa']=='Segtruck']['placa'].count()
placas_stcoop = df_total[df_total['empresa']=='Stcoop']['placa'].count()
placas_viavante = df_total[df_total['empresa']=='Viavante']['placa'].count()

print("Total de placas:", f"{placas_totais:,}".replace(",", "."))
print("Placas Segtruck:", f"{placas_segtruck:,}".replace(",", "."))
print("Placas Stcoop:", f"{placas_stcoop:,}".replace(",", "."))
print("Placas Viavante:", f"{placas_viavante:,}".replace(",", "."))

Total de placas: 32.242
Placas Segtruck: 2.215
Placas Stcoop: 4.993
Placas Viavante: 25.034


# ENVIANDO POR E-MAIL

In [ ]:
import win32com.client as win32
import pythoncom
from datetime import datetime

pythoncom.CoInitialize()

date = date.strftime('%d/%m/%Y')

outlook = win32.Dispatch("Outlook.Application")
email = outlook.CreateItem(0)
email.To = "dados13@grupounus.com.br"#; supervisao.dados@grupounus.com.br; dados03@grupounus.com.br"
email.Subject = f'[PLACAS ATIVAS POR EMPRESA] - DATA:{date}'
email.HTMLBody = f"""
    <html>
    <body>
        <p>Prezados(as),</p>
        <p>A seguir, a quantidade de placas ativas por empresa, no dia {date}:</p>
        <ul>
            <li><b>SEGTRUCK:</b> {f"{placas_segtruck:,}".replace(",", ".")}</li>
            <li><b>STCOOP:</b> {f"{placas_stcoop:,}".replace(",", ".")}</li>
            <li><b>VIAVANTE:</b> {f"{placas_viavante:,}".replace(",", ".")}</li>
            <li><b>TOTAL:</b> {f"{placas_totais:,}".replace(",", ".")}</li>
        </ul>
        <p>Atenciosamente,</p>
        <p>Equipe Análise de Dados - Grupo Unus</p>
        <p><i>Este é um e-mail automático, por favor não responda</i></p>
    </body>
    </html>
"""

email.Send()
print("Email enviado com sucesso!")

Email enviado com sucesso!
